<a href="https://colab.research.google.com/github/gogosean/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE

import pandas as pd


df=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data', header=None)


In [140]:
df.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
df.columns = ['party','handicapped','water','adoption','physician','aid','religious','ban','contras','missile','immigration','sunfuels','education','superfund','crime','duty','export']

In [142]:
df.head(5)

,party,handicapped,water,adoption,physician,aid,religious,ban,contras,missile,immigration,sunfuels,education,superfund,crime,duty,export
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
df2= df.reset_index()

In [144]:
df2.columns

Index(['index', 'party', 'handicapped', 'water', 'adoption', 'physician',
       'aid', 'religious', 'ban', 'contras', 'missile', 'immigration',
       'sunfuels', 'education', 'superfund', 'crime', 'duty', 'export'],
      dtype='object')

In [145]:
df2= df2.drop(['index'], axis=1)
df2.columns

Index(['party', 'handicapped', 'water', 'adoption', 'physician', 'aid',
       'religious', 'ban', 'contras', 'missile', 'immigration', 'sunfuels',
       'education', 'superfund', 'crime', 'duty', 'export'],
      dtype='object')

In [0]:
import numpy as np



In [147]:
#def apply_code(string):
# if (string == "?") :
#     return np.nan()

df2= df2.replace('?',np.NaN)

df2.head(5)

,party,handicapped,water,adoption,physician,aid,religious,ban,contras,missile,immigration,sunfuels,education,superfund,crime,duty,export
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [0]:
def apply_code(string): 
 if (string == "y") :
     return int(1)
 if (string=="n"):
     return int(0)
  

In [0]:
#df2['handicapped']=df2['handicapped'].apply(apply_code)

In [150]:
df2.head(5)
df2.columns

Index(['party', 'handicapped', 'water', 'adoption', 'physician', 'aid',
       'religious', 'ban', 'contras', 'missile', 'immigration', 'sunfuels',
       'education', 'superfund', 'crime', 'duty', 'export'],
      dtype='object')

In [151]:

vars= list(df2.columns.values)

vars.remove('party')
print(vars)

 


['handicapped', 'water', 'adoption', 'physician', 'aid', 'religious', 'ban', 'contras', 'missile', 'immigration', 'sunfuels', 'education', 'superfund', 'crime', 'duty', 'export']


In [0]:
for col in vars:
   df2[col]=df2[col].apply(apply_code)

In [153]:
df2.columns

Index(['party', 'handicapped', 'water', 'adoption', 'physician', 'aid',
       'religious', 'ban', 'contras', 'missile', 'immigration', 'sunfuels',
       'education', 'superfund', 'crime', 'duty', 'export'],
      dtype='object')

In [154]:
df2.head(5)


,party,handicapped,water,adoption,physician,aid,religious,ban,contras,missile,immigration,sunfuels,education,superfund,crime,duty,export
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [156]:
rep = df2[df2['party'] == 'republican']

dem = df2[df2['party'] == 'democrat']
print(rep.head())
print(dem.head())

         party  handicapped  water  adoption  ...  superfund  crime  duty  export
0   republican          0.0    1.0       0.0  ...        1.0    1.0   0.0     1.0
1   republican          0.0    1.0       0.0  ...        1.0    1.0   0.0     NaN
7   republican          0.0    1.0       0.0  ...        1.0    1.0   NaN     1.0
8   republican          0.0    1.0       0.0  ...        1.0    1.0   0.0     1.0
10  republican          0.0    1.0       0.0  ...        1.0    1.0   0.0     0.0

[5 rows x 17 columns]
      party  handicapped  water  adoption  ...  superfund  crime  duty  export
2  democrat          NaN    1.0       1.0  ...        1.0    1.0   0.0     0.0
3  democrat          0.0    1.0       1.0  ...        1.0    0.0   0.0     1.0
4  democrat          1.0    1.0       1.0  ...        1.0    1.0   1.0     1.0
5  democrat          0.0    1.0       1.0  ...        1.0    1.0   1.0     1.0
6  democrat          0.0    1.0       0.0  ...        NaN    1.0   1.0     1.0

[5 rows x 

In [159]:
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

for col in vars:
  print('    ')
  print('--------------------')
  print(col)
  print('democrat')
  print(dem[col].mean())
  print('republican')
  print(rep[col].mean())
  print(ttest_ind(rep[col], dem[col], nan_policy='omit'))
  print('--------------------')
  print('    ')
   #df2[col]=df2[col].apply(apply_code)

    
--------------------
handicapped
democrat
0.6046511627906976
republican
0.18787878787878787
Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)
--------------------
    
    
--------------------
water
democrat
0.502092050209205
republican
0.5067567567567568
Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)
--------------------
    
    
--------------------
adoption
democrat
0.8884615384615384
republican
0.13414634146341464
Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)
--------------------
    
    
--------------------
physician
democrat
0.05405405405405406
republican
0.9878787878787879
Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)
--------------------
    
    
--------------------
aid
democrat
0.21568627450980393
republican
0.9515151515151515
Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)
--------------------
    
    
--------------------
re

**Solutions:**









2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
**handicapped-infants**
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
**religious-groups-in-schools:**
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)
**water-project-cost-sharing**

